In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import (
    ElementClickInterceptedException,
    NoSuchElementException,
    StaleElementReferenceException,
    TimeoutException
)
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Chrome options setup
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")

# Path to your ChromeDriver
driver_path = "/usr/local/bin/chromedriver" 
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

wait = WebDriverWait(driver, 20)  # Wait instance

try:
    url = "https://www.tradingview.com/markets/stocks-usa/market-movers-all-stocks/"
    driver.get(url)
    time.sleep(5)  # Initial wait for page load

    # Click "Load More" until disabled or unavailable
    while True:
        try:
            load_more_button = wait.until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="js-category-content"]/div[2]/div/div[4]/div[3]/button'))
            )
            aria_disabled = load_more_button.get_attribute("aria-disabled")
            if aria_disabled == "true":
                print("Load More button disabled, all data loaded.")
                break

            driver.execute_script("arguments[0].click();", load_more_button)
            time.sleep(3)
        except (TimeoutException, NoSuchElementException, ElementClickInterceptedException):
            print("Load More button not found or not clickable anymore.")
            break

    # Wait for table rows to be present
    wait.until(EC.presence_of_all_elements_located((By.XPATH, "//table//tbody/tr")))

    # Scrape the rows
    data = []
    rows = driver.find_elements(By.XPATH, "//table//tbody/tr")

    for i in range(len(rows)):
        try:
            row = driver.find_elements(By.XPATH, "//table//tbody/tr")[i]
            cells = row.find_elements(By.TAG_NAME, "td")
            row_data = [cell.text for cell in cells]
            data.append(row_data)
        except StaleElementReferenceException:
            print(f"Stale element at row {i}, skipping this row.")

    columns = [
        "Symbol", "Price", "Change %", "Volume", "Rel Volume", "Market Cap",
        "P/E", "EPS (TTM)", "EPS Growth (YoY)", "Div Yield %", "Sector", "Analyst Rating"
    ]

    filtered_data = [row for row in data if len(row) == len(columns)]

    if len(filtered_data) < len(data):
        print(f"Warning: {len(data) - len(filtered_data)} rows skipped due to column mismatch.")

    if not filtered_data:
        raise Exception("No valid data rows matched expected column structure.")

    # Save as CSV file with try-except inside this try
    df = pd.DataFrame(filtered_data, columns=columns)
    save_path = "/Users/farazahmed/Documents/data2.csv"

    try:
        df.to_csv(save_path, index=False)
        print(f"Data successfully saved to '{save_path}'.")
    except PermissionError:
        print(f"Permission error: please close the file or check write access to '{save_path}'")
    except Exception as e:
        print(f"Error saving file: {e}")

finally:
    driver.quit()


Load More button disabled, all data loaded.
Data successfully saved to '/Users/farazahmed/Documents/data2.csv'.
